In [ ]:

    # Step 1: Set Access Key in Spark Configuration
spark.conf.set(
    "fs.azure.account.key.strgsk.blob.core.windows.net",
    "+LOoFwx9bDT71IIpSIk0H6+C3Eclj2ZEfvzAGTPVoExZHTcUewPy/uQ0jqO9ExhY+I+jptSHh7+S+AStt20MQA=="
)
 
# Step 2: Define file path (WASBS path)
file_location = "wasbs://week8container@strgsk.blob.core.windows.net/yellow-trip-data/nyc_tripdata_sample.csv"
 
# Step 3: Load CSV File
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(file_location)
 
# Step 4: Display Data
df.show(5)
df.printSchema()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+
|       1| 2020-01-01 00:00:00|  2020-01-01 00:30:00|              1|         1.47|         1|                 Y|         165|          79|           1|       17.5| 0.74|   0.14|      2.84|        1.81|                 0.44|       23.47|
|       2| 2020-01-01 01:00:00|  2020-01-01 01:3

**Query 1** - <span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);">Add a column named as ""Revenue"" into dataframe which is the sum of the below columns 'Fare_amount','Extra','MTA_tax','Improvement_surcharge','Tip_amount','Tolls_amount','Total_amount'</span>

In [ ]:
from pyspark.sql.functions import col, sum as _sum
df = df.withColumn("Revenue",
    col("fare_amount") +
    col("extra") +
    col("mta_tax") +
    col("improvement_surcharge") +
    col("tip_amount") +
    col("tolls_amount") +
    col("total_amount")
)

df.select("Revenue", "fare_amount", "extra", "mta_tax", "tip_amount","tolls_amount", "total_amount", ).show(5)


+------------------+-----------+-----+-------+----------+------------+------------+
|           Revenue|fare_amount|extra|mta_tax|tip_amount|tolls_amount|total_amount|
+------------------+-----------+-----+-------+----------+------------+------------+
|             46.94|       17.5| 0.74|   0.14|      2.84|        1.81|       23.47|
|120.25999999999999|      43.32| 0.86|   0.41|     11.75|        3.62|       60.13|
|              37.6|       5.01| 1.69|   0.14|     10.73|         1.0|        18.8|
|             33.76|        5.8| 0.54|   0.41|       8.9|        1.02|       16.88|
|             92.38|      32.11| 0.31|   0.27|     13.44|        0.05|       46.19|
+------------------+-----------+-----+-------+----------+------------+------------+
only showing top 5 rows



**Query 2 -**  <span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);">&nbsp;Increasing count of total passengers in New York City by area</span>

In [ ]:
df.groupBy("PULocationID").agg(_sum("passenger_count").alias("TotalPassengers")).orderBy("TotalPassengers", ascending=False).show(10)

+------------+---------------+
|PULocationID|TotalPassengers|
+------------+---------------+
|         227|            600|
|          59|            560|
|          17|            520|
|          98|            520|
|          40|            440|
|          29|            400|
|         215|            400|
|         108|            320|
|         164|            320|
|           3|            320|
+------------+---------------+
only showing top 10 rows



<span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);"><b>Query 3</b>. - Realtime Average fare/total earning amount earned by 2 vendors</span>

In [ ]:
df.groupBy("VendorID").agg({"total_amount": "avg"}).withColumnRenamed("avg(total_amount)", "AverageEarnings").show()


+--------+------------------+
|VendorID|   AverageEarnings|
+--------+------------------+
|       1| 39.20774834437135|
|       2|38.514362416107296|
+--------+------------------+



<span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);"><b>Query 4. </b>- Moving Count of payments made by each payment mode</span>

In [ ]:
df.groupBy("payment_type").count().orderBy("count", ascending=False).show()


+------------+-----+
|payment_type|count|
+------------+-----+
|           4| 2720|
|           2| 2640|
|           5| 2400|
|           1| 2240|
|           3| 2000|
+------------+-----+



<span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);"><b>Query 5. </b>- Highest two gaining vendor's on a particular date with no of passenger and total distance by cab&nbsp;</span>

In [ ]:
from pyspark.sql.functions import to_date

df_with_date = df.withColumn("trip_date", to_date("tpep_pickup_datetime"))
df_with_date.groupBy("trip_date", "VendorID").agg(
    _sum("total_amount").alias("TotalEarnings"),
    _sum("passenger_count").alias("TotalPassengers"),
    _sum("trip_distance").alias("TotalDistance")
).orderBy("TotalEarnings", ascending=False).show(2)


+----------+--------+------------------+---------------+------------------+
| trip_date|VendorID|     TotalEarnings|TotalPassengers|     TotalDistance|
+----------+--------+------------------+---------------+------------------+
|2020-01-11|       1| 28127.19999999997|           1760|3619.6000000000067|
|2020-01-08|       1|27503.599999999973|           1560|3916.0000000000114|
+----------+--------+------------------+---------------+------------------+
only showing top 2 rows



<span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);"><b>Query 6</b>. - Most no of passenger between a route of two location.</span>

In [ ]:
df.groupBy("PULocationID", "DOLocationID") \
  .agg(_sum("passenger_count").alias("TotalPassengers")) \
  .orderBy("TotalPassengers", ascending=False).show(1)


+------------+------------+---------------+
|PULocationID|DOLocationID|TotalPassengers|
+------------+------------+---------------+
|          57|          11|            160|
+------------+------------+---------------+
only showing top 1 row



<span style="color: rgb(75, 75, 75); font-family: Manrope, sans-serif; font-size: 13px; text-align: justify; background-color: rgb(255, 255, 255);"><b>Query 7. </b>- Get top pickup locations with most passengers in last 5/10 seconds."</span>

In [ ]:
latest_time = df.agg(max("tpep_pickup_datetime")).first()[0]
window_start = latest_time - expr("INTERVAL 10 seconds")

df.filter((col("tpep_pickup_datetime") > window_start) & (col("tpep_pickup_datetime") <= latest_time))\
  .groupBy("PULocationID")\
  .agg(sum("passenger_count").alias("Recent_Passengers"))\
  .orderBy(desc("Recent_Passengers"))\
  .show()

+------------+---------------+
|PULocationID|TotalPassengers|
+------------+---------------+
|         134|             80|
+------------+---------------+



**CSV to Paraquet in Blob Storage**

In [ ]:
output_path = "wasbs://week8container@strgsk.blob.core.windows.net/output_parquet/"
 
df.write.mode("overwrite").parquet(output_path)